In [ ]:
import os, sys
root_path = os.path.abspath('.').split('jupyters')[0]
sys.path.append(root_path)

In [ ]:
import json
import argparse
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from data_loader import load_data
from models.cnn_geo import CNN_geo

from debug_tools import data, train, visualize
from utils import image
import geo_transform as tps
import CNNgeo_debug

# 1. Read configuration file

In [ ]:
with open('../overfit.json') as fp:
    config = json.load(fp)
split = 'val'

In [ ]:
print(config)

In [ ]:
print(config['train'])

In [ ]:
config['backbone'] = 'vgg16'
config['train']['epochs'] = 200

In [ ]:
print("backbone : ", config['backbone'])
print("n_examples : ", config['train']['n_examples'])
print("learning_rate : ", config['train']['learning_rate'])
print("batch_size : ", config['train']['batch_size'])
print("epoch num : ", config['train']['epochs'])

# 2. Train model on the config parameters

In [ ]:
model = CNNgeo_debug.train(config)

In [ ]:
model.summary()

# 3. Debug

In [ ]:
datasets = load_data(['train'], config)
ds = datasets['train'].batch(1)

In [ ]:
for image_A, image_B, parameters in ds.take(1):
    image_A = image_A.numpy()
    image_B = image_B.numpy()
    parameters = parameters.numpy()
#image_B_hat = np.ones([1, 64, 64, 3])
pred, score = model(image_A, image_B)

In [ ]:
print("parameter shape", pred.shape)
print("score shape", score.shape)
print(image_A.shape)

## 3.1. 입력 영상에 따른 모델 추정 확인

### case 1) 임의의 기하관계를 갖는 영상 쌍 (image A, image B)

In [ ]:
for image_A, image_B, parameters in ds.take(1):
    image_A = image_A.numpy()
    image_B = image_B.numpy()
    parameters = parameters.numpy()
pred, score = model(image_A, image_B)

print("compare gt : {} and \n pred : {}".format(parameters, pred))
loss = tf.reduce_sum(tf.keras.losses.MSE(pred, parameters), axis=1)
print("loss : {}".format(loss))

pred = pred.numpy()
image_C = list(map(lambda x : image.synthesize_image(x[0], x[1], (64, 64), bbox=None, pad_ratio=None),
                   zip(image_A.copy(), pred.copy())))    
image_C = np.array(image_C)
visualize.show_TPS_image([image_A, image_B, image_C], [np.ones_like(parameters), parameters, pred])    

pred1 = pred.copy()

### case 2) 임의의 다른 기하관계를 갖는 영상 쌍 (image A, image B)

In [ ]:
for image_A, image_B, parameters in ds.take(1):
    image_A = image_A.numpy()
    image_B = image_B.numpy()
    parameters = parameters.numpy()
pred, score = model(image_A, image_B)

print("compare gt : {} and \n pred : {}".format(parameters, pred))
loss = tf.reduce_sum(tf.keras.losses.MSE(pred, parameters), axis=1)
print("loss : {}".format(loss))

pred = pred.numpy()
image_C = list(map(lambda x : image.synthesize_image(x[0], x[1], (64, 64), bbox=None, pad_ratio=None),
                   zip(image_A.copy(), pred.copy())))    
image_C = np.array(image_C)
visualize.show_TPS_image([image_A, image_B, image_C], [np.ones_like(parameters), parameters, pred])    

pred2 = pred.copy()

### case 3) Image A와 연관 없는 image B(백색 사진)의 영상 쌍 

In [ ]:
image_B = np.ones([1, 64, 64, 3])
pred, score = model(image_A, image_B)

print("compare gt : {} and \n pred : {}".format(parameters, pred))
loss = tf.reduce_sum(tf.keras.losses.MSE(pred, parameters), axis=1)
print("loss : {}".format(loss))

pred = pred.numpy()
image_C = list(map(lambda x : image.synthesize_image(x[0], x[1], (64, 64), bbox=None, pad_ratio=None),
                   zip(image_A.copy(), pred.copy())))    
image_C = np.array(image_C)
visualize.show_TPS_image([image_A, image_B, image_C], [np.ones_like(parameters), parameters, pred])    

pred3 = pred.copy()

In [ ]:
print(pred1, "\n", pred2, "\n", pred3)

case 1)와 case 2)는 동일한 영상을 다른 기하관계를 사용하여 두 개의 영상쌍(image pair)을 만든 경우를 말한다. 모델이 정상적이라면 각 영상쌍의 기하관계를 추정할 수 있을 것이다. 그러나 두 영상쌍이 다른 기하관계를 갖음에도 동일한 모션파라미터를 추정하였다. case 3)에서는 전혀 연관 없는 영상을 image B로 사용하였다. image B는 어떤 특징 정보도 없는 백색 사진임에도 불구하고 모델은 case 1)과 case 2)에서와 같은 모션파라미터를 추정하였다.

In [ ]:
model.summary()

## 3.2 Correlation 분포에 따른 Regressor 예측

3.1의 case 1, 2, 3에서 Regressor은 같은 모션파라미터를 예측했다. 명확한 이유를 파악하기 위해 입력 correlation(matching scores)를 제어하며 spatial parameter regressor의 출력을 분석했다.

In [ ]:
x = np.zeros([1, 16, 16, 16, 16]) # all correlations are zero
y = model.layers[2](x)
print(y)

In [ ]:
x = np.ones([1, 16, 16, 16, 16]) # all correlations are one
y = model.layers[2](x)
print(y)

In [ ]:
x = np.random.normal(loc=0.5, scale=0.0, size=[1, 16, 16, 16, 16])
y = model.layers[2](x)
print(y)

In [ ]:
x = np.random.normal(loc=0.5, scale=0.3, size=[1, 16, 16, 16, 16])
y = model.layers[2](x)
print(y)

위 실험의 결과를 토대로 확인할 수 있는 것은 regressor의 출력은 입력 correlation의 크기가 아닌 편차에 따라 달라진다는 것이다. correlation의 분포의 표준편차를 변화시키며 출력의 변화 정도를 추정해주었다.

In [ ]:
_range = np.linspace(0, 1.0, 50)
print(_range)

In [ ]:
variations_via_std = []
variations_via_mean = []
for std in _range:
    variations = []
    for i in range(1000):
        x = np.random.normal(loc=0.5, scale=std, size=[1, 16, 16, 16, 16])
        _y = model.layers[2](x)
        variations.append(_y)
    variations_via_std.append(np.sum(np.std(np.array(variations), axis = 0)))
    
for mean in _range:
    variations = []
    for i in range(1000):
        x = np.random.normal(loc=mean, scale=0.1, size=[1, 16, 16, 16, 16])
        _y = model.layers[2](x)
        variations.append(_y)
    variations_via_mean.append(np.sum(np.std(np.array(variations), axis = 0)))

In [ ]:
variations

In [ ]:
plt.ylabel("variations of motion parameters")
plt.xlabel("via sigma, mean. \n correlations are sampled from N(0.5, sigma), N(mean, 0.1)")
plt.plot(_range, variations_via_std, label='via sigma')
plt.plot(_range, variations_via_mean, label='via mean')
plt.legend()
plt.show()

1. VGG16을 통한 correlations
2. 시각화 방식 개선
3. tentative penalty

## b. 특징추출기의 정상 동작 검증

In [ ]:
np.set_printoptions(formatter = {"float_kind" : lambda x : "{0:0.4f}".format(x)} )
#np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})

### 동일 영상에 대한 correlation

In [ ]:
for image_A, image_B, parameters in ds.take(1):
    image_A = image_A.numpy()
    image_B = image_B.numpy()
    parameters = parameters.numpy()
pred, score = model(image_A, image_A)

score = score.numpy()[0]
parameters = parameters[0]
image_A = image_A[0]
image_B = image_B[0]

print(score.shape, parameters.shape, image_A.shape)

In [ ]:
def get_matching_grid_from_B(parameters, center_point):
    src_points = np.array([[0.0, 0.0], [0.5, 0.0], [1.0, 0.0],
                                   [0.0, 0.5], [0.5, 0.5], [1.0, 0.5],
                                   [0.0, 1.0], [0.5, 1.0], [1.0, 1.0]])

    dst_points = src_points +parameters

    theta = tps.tps_theta_from_points(src_points, dst_points, reduced=True)
    dshape = (64, 64)
    grid = tps.tps_grid(theta, dst_points, dshape)
    mapx, mapy = tps.tps_grid_to_remap(grid, (64, 64))
    points = np.concatenate([mapy[:,:,np.newaxis], mapx[:,:,np.newaxis]], axis=2)
    #print("points :", points)
    center_point = np.array(center_point)
    #print("grid center : ", center_point)
    center_point = center_point[np.newaxis, np.newaxis, :]
    distance = np.sum(np.power((points - center_point), 2), axis=2)
    #print("distance : ", distance)
    ri, ci = distance.argmin()//distance.shape[1], distance.argmin()%distance.shape[1]
    return (ri, ci)

In [ ]:
def score_test(grid_coord, score, parameters, images, grid_shape=(16,16)):
    image_A, image_B = images
    score = score[grid_coord[0], grid_coord[1]]
    H, W, C = image_A.shape
    grid_size = H/grid_shape[0], W/grid_shape[1]

    start_pix_h = int(grid_size[0]*grid_coord[0])
    end_pix_h = int(grid_size[0]*(1+grid_coord[0]))
    start_pix_w = int(grid_size[1]*grid_coord[1])
    end_pix_w = int(grid_size[1]*(1+grid_coord[1]))
    
    drawn_grid_image_A = image_A.copy()
    drawn_grid_image_A[start_pix_h:end_pix_h, start_pix_w:end_pix_w] = (0,0,0)   
    
    expected_drawn_grid_image_B = image_B.copy()
    grid_center_A = (int(start_pix_h+grid_size[0]/2), int(start_pix_w+grid_size[1]/2))
    grid_center_B = get_matching_grid_from_B(parameters, grid_center_A)
    start_pix_h = int(grid_center_B[0] - grid_size[0]/2)
    start_pix_w = int(grid_center_B[1] - grid_size[1]/2)
    end_pix_h = int(grid_center_B[0] + grid_size[0]/2)
    end_pix_w = int(grid_center_B[1] + grid_size[1]/2)
    expected_drawn_grid_image_B[start_pix_h:end_pix_h, start_pix_w:end_pix_w] = (0,0,0)
    
    drawn_grid_image_B = image_B.copy()
    max_correlation_grid_index = score.argmax()//score.shape[1], score.argmax()%score.shape[1]
    start_pix_h = int(grid_size[0]*max_correlation_grid_index[0])
    end_pix_h = int(grid_size[0]*(1+max_correlation_grid_index[0]))
    start_pix_w = int(grid_size[1]*max_correlation_grid_index[1])
    end_pix_w = int(grid_size[1]*(1+max_correlation_grid_index[1]))

    drawn_grid_image_B[start_pix_h:end_pix_h, start_pix_w:end_pix_w] = (0,0,0)
    
    print("top 10 correlation values in descending : ", np.sort(score.flatten())[::-1][:10])
    print("standard deviation : {}".format(np.std(score)))
    return drawn_grid_image_A, expected_drawn_grid_image_B, drawn_grid_image_B, score

In [ ]:
drawn_image_A, expected_drawn_grid_image_B, drawn_grid_image_B, patch_score= score_test(grid_coord=(5,10), 
                                                                            score=score, 
                                                                            parameters=np.zeros([9,2]), 
                                                                            images=(image_A, image_A))

In [ ]:
# n, bins, patches = plt.hist(patch_score, bins=10, density=False, facecolor='blue', alpha=0.5)
# plt.xlabel('X bins')
# plt.ylabel('Frequency')
# plt.title('Histogram of correlations')


fig = plt.figure()

ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
ax1.imshow(drawn_image_A)
ax2.imshow(expected_drawn_grid_image_B)
ax3.imshow(drawn_grid_image_B)
plt.show()
n, bins, patches = plt.hist(patch_score, bins=5, density=False, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
drawn_image_A.shape
expected_drawn_grid_image_B.shape
drawn_grid_image_B.shape

### b.1 ) 동일한 사진의 경우

In [ ]:
for i in range(16):
    drawn_image_A, expected_drawn_grid_image_B, drawn_grid_image_B, patch_score= score_test(grid_coord=(i,i), 
                                                                                score=score, 
                                                                                parameters=np.zeros([9,2]), 
                                                                                images=(image_A, image_A))   
    fig = plt.figure()
    
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)

    ax1.imshow(drawn_image_A)
    ax2.imshow(expected_drawn_grid_image_B)
    ax3.imshow(drawn_grid_image_B)

    plt.show()
    n, bins, patches = plt.hist(patch_score, bins=5, density=False, facecolor='blue', alpha=0.5)
    plt.axis([0.0, 0.070, 0, 20])
    plt.show()

### b.2 ) 다른 사진의 경우

In [ ]:
for image_A, image_B, parameters in ds.take(1):
    image_A = image_A.numpy()
    image_B = image_B.numpy()
    parameters = parameters.numpy()
pred, score = model(image_A, image_A)

score = score.numpy()[0]
parameters = parameters[0]
image_A = image_A[0]
image_B = image_B[0]

print(score.shape, parameters.shape, image_A.shape)

In [ ]:
for i in range(16):
    drawn_image_A, expected_drawn_grid_image_B, drawn_grid_image_B, patch_score = score_test(grid_coord=(i,i), 
                                                                                score=score, 
                                                                                parameters=parameters, 
                                                                                images=(image_A, image_B))   
    fig = plt.figure()

    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)

    ax1.imshow(drawn_image_A)
    ax2.imshow(expected_drawn_grid_image_B)
    ax3.imshow(drawn_grid_image_B)

    plt.show()
    n, bins, patches = plt.hist(patch_score, bins=5, density=False, facecolor='blue', alpha=0.5)
    plt.axis([0.055, 0.070, 0, 30])
    plt.show()